In [1]:
# This block is from https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

import seaborn as sns #collection of functions for data visualization
print("seaborn version: {}". format(sns.__version__))

from sklearn.preprocessing import OneHotEncoder #OneHot Encoder

#misc libraries
import random
import time


#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)


Python version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.0.1
matplotlib version: 3.1.3
NumPy version: 1.18.1
SciPy version: 1.4.1
IPython version: 7.12.0
scikit-learn version: 0.22.1
seaborn version: 0.10.0
-------------------------


## Load the files

In [8]:
train_raw = pd.read_csv('data/train.csv')
test_raw = pd.read_csv('data/test.csv')
sample_raw = pd.read_csv('data/sample_submission.csv')

In [9]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [20]:
def null_p (column):
    num_rows = train_raw.shape[0]
    null_percent = (column.isnull().sum() /num_rows)*100
    return null_percent

In [21]:
#drop columns with more than null_bar% of null data
def drop_null_col (df, null_bar):
    col_ls = []
    for col in df.columns:

        if null_p(df[col]) > null_bar:
            col_ls.append(col)
        
    return col_ls

In [22]:
cols_drop = drop_null_col(train_raw, 40)

In [23]:
cols_drop

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [26]:
train_data_dnull = train_raw.copy().drop(cols_drop, axis = 1)

In [29]:
def null_p_df (df):
    p_ls = []
    for col in df.columns:
        n_percent = null_p(df[col])
        p_ls.append(n_percent)
    np_df = pd.DataFrame({'Column Name': df.columns, "Null Percent": p_ls})
    return np_df

In [30]:
np_df = null_p_df(train_data_dnull)

In [37]:
np_df.sort_values(by = 'Null Percent',  ascending=False).head(20)

,Column Name,Null Percent
3,LotFrontage,17.739726
56,GarageType,5.547945
62,GarageCond,5.547945
57,GarageYrBlt,5.547945
58,GarageFinish,5.547945
61,GarageQual,5.547945
34,BsmtFinType2,2.602740
31,BsmtExposure,2.602740
32,BsmtFinType1,2.534247
30,BsmtCond,2.534247
